<a href="https://colab.research.google.com/github/SiraDD/TuringProject/blob/main/IntroToSeminarCases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.0 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

In [3]:
# importing values from excel file
df = pd.read_excel('Railway services-2024.xlsx')
df = df.iloc[:-3]
indexes = df['Trip'].tolist()
demand = df['Demand(μ)'].tolist()
line = df['Line'].tolist()
demand_stdDev = df['Demand(σ)'].tolist()

Exercise 3 - Formulation 1 (Cross-sections + train types)

In [ ]:
# create model
m1 = gp.Model("first formulation ex 3")

# create variables
N = m1.addVars(2, 200, vtype=GRB.INTEGER) #N_{u,c} variables (2 types & 200 cross-sections) (0 = OC & 1 = OH)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict = {}
capacityCoefficients = [620, 420]
capacityDict = {}
lengthCoefficients = [100, 70]
lengthDict = {}
qty1Coefficients = [1, -1.25]
qty1Dict = {}
qty2Coefficients = [-1.25, 1]
qty2Dict = {}

for u in range(2):
  for c in range(200):
    costDict[(u, c)] = costCoefficients[u]
    capacityDict[(u, c)] = capacityCoefficients[u]
    lengthDict[(u, c)] = lengthCoefficients[u]
    qty1Dict[(u, c)] = qty1Coefficients[u]
    qty2Dict[(u, c)] = qty2Coefficients[u]

In [ ]:
# Defining Objective Function
m1.setObjective(N.prod(costDict))

# Generating Constraints
m1.addConstrs(N.prod(capacityDict, '*', c) >= demand[c]  for c in range(200))
m1.addConstr(N.prod(qty1Dict) <= 0)
m1.addConstr(N.prod(qty2Dict) <= 0)
m1.addConstrs(N.prod(lengthDict, '*', c) <= lengthRequirement[c] for c in range(200))
m1.addConstrs(N[u, c] >= 0 for u in range(2) for c in range(200))

m1.optimize()

In [ ]:
# Solve LP relaxation
r1 = m1.relax()
r1.optimize()

Exercise 3 - Formulation 2 (Compositions + Cross-sections)

In [ ]:
# Create set of compositions - denoted as a pair of values (OC, OH), where OC is the number of OC rolling stock units and OH is the number of OH rolling stock units
compositions = []
minDemand = min(demand)

for OC in range(4):
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

print(compositions)

In [ ]:
# Create model
m2 = gp.Model("second formulation ex 3")

# create variables
X = m2.addVars(200, 10, vtype=GRB.BINARY) #X_{c,p} variables (200 cross-sections & 10 compositions)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
lengthCoefficients = [100, 70]
lengthDict2 = {}
qty1Coefficients = [1, -1.25]
qty1Dict2 = {}
numOfType1Trains = {}
numOfType2Trains = {}
qty2Coefficients = [-1.25, 1]
qty2Dict2 = {}

for composition in range(len(compositions)):
  for c in range(200):
    costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
    capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
    lengthDict2[(c, composition)] = lengthCoefficients[0] * compositions[composition][0] + lengthCoefficients[1] * compositions[composition][1]
    qty1Dict2[(c, composition)] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
    qty2Dict2[(c, composition)] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

In [ ]:
# Defining Objective Function
m2.setObjective(X.prod(costDict2))

# Generating Constraints
m2.addConstrs(X.prod(capacityDict2, c, '*') >= demand[c]  for c in range(200))
m2.addConstr(X.prod(qty1Dict2) <= 0)
m2.addConstr(X.prod(qty2Dict2) <= 0)
m2.addConstrs(X.prod(lengthDict2, c, '*') <= lengthRequirement[c] for c in range(200))
m2.addConstrs(X.sum(c, '*') == 1 for c in range(200))

m2.optimize()

In [ ]:
# Solve LP relaxation
r2 = m2.relax()
r2.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 602 rows, 2000 columns and 10000 nonzeros
Model fingerprint: 0xd3769065
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [2e+05, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve time: 0.02s
Presolved: 602 rows, 2000 columns, 10000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+07   4.945719e+03   0.000000e+00      0s
     343    6.2616490e+07   0.000000e+00   0.000000e+00      0s

Solved in 343 iterations and 0.04 seconds (0.01 work units)
Optimal objective  6.261648954e+07


Exercise 4 - Stochastic Demand

In [4]:
# Create set of compositions - denoted as a pair of values (OC, OH), where OC is the number of OC rolling stock units and OH is the number of OH rolling stock units
compositions = []
minDemand = min(demand)

for OC in range(4):
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

# Create stochastic demand
stochasticDemands = [[0 for days in range(250)] for c in range(200)] # stochasticDemands[c][days]
stochasticDemands = np.array(stochasticDemands)

for c in range(200):
  demandMean = demand[c]
  demandStDev = demand_stdDev[c]
  for day in range(250):
    stochasticDemands[c][day] = np.random.normal(demandMean, demandStDev)

params = {
"WLSACCESSID": '9f6bda1d-2c0b-49c7-96a8-f172b568e9e4',
"WLSSECRET": '3db7e79a-7091-4079-9dc3-c9c1382b1c4e',
"LICENSEID": 2498504,
}
env = gp.Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498504
Academic license 2498504 - for non-commercial use only - registered to 61___@eur.nl


In [5]:
# Create model
m3 = gp.Model("Exercise 4 second formulation", env = env)

# Create variables
X2 = m3.addMVar((200, 10), vtype=GRB.BINARY) #X_{c,p} variables (200 cross-sections & 10 compositions)
Y = m3.addMVar((200, 250), vtype=GRB.BINARY) #Y_{c, d} variables (200 cross-sections & 250 days)

# Create coefficient vectors
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
capacityCoefficients = [620, 420]
lengthCoefficients = [100, 70]
qty1Coefficients = [1, -1.25]
qty2Coefficients = [-1.25, 1]

costVector = np.zeros(10)
capacityVector  = np.zeros(10)
lengthVector = np.zeros(10)
qty1Vector = np.zeros(10)
qty2Vector = np.zeros(10)

for composition in range(len(compositions)):
  costVector[composition] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
  capacityVector[composition] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
  lengthVector[composition] = lengthCoefficients[0] * compositions[composition][0] + lengthCoefficients[1] * compositions[composition][1]
  qty1Vector[composition] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
  qty2Vector[composition] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

In [ ]:
# Defining Objective Function
m3.setObjective(X2.sum(0) @ costVector)

for d in range(250):
  for c in range(200):
    m3.addGenConstrIndicator(Y[c][d], 1, X2[c] @ capacityVector, GRB.GREATER_EQUAL, stochasticDemands[c][d])

# Remaining Constraints
m3.addConstr(Y.sum() / 50000 >= 0.81)
m3.addConstrs(Y.sum(1)[c] / 250 >= 0.5 for c in range(200))
m3.addConstr(X2.sum(0) @ qty1Vector <= 0)
m3.addConstr(X2.sum(0) @ qty2Vector <= 0)
m3.addConstrs(X2[c] @ lengthVector <= lengthRequirement[c] for c in range(200))
m3.addConstrs(X2.sum(1)[c] == 1 for c in range(200))

m3.optimize()